In [13]:
import sys
# Для начала что такое алгоритм дейкстры. Это алгоритм поиска кратчайших путей между узлами взвешенного ориентированного графа
# примерное обьяснение в gif https://upload.wikimedia.org/wikipedia/commons/5/57/Dijkstra_Animation.gif

# От начальной вершины алгоритм присваивает остальным вершинам(nodes) кратчайшие пути, при этом если есть альтернатива
# он сравнивает числовое значение путей и оставляет минимальное, тем самым ища кратчайший путь.

# Для начала чтобы реализовать алгоритм создадим класс графы, в котором обозначим различные функции коммуникации с созданным графом
# список названий вершин - nodes и словарь со словарями(реализованный граф) init_graph
# Список: ["Reykjavik", "Oslo", "Moscow", "London", "Rome", "Berlin", "Belgrade", "Athens"]
# Словарь: {'Reykjavik': {'Oslo': 5, 'London': 4},
#              'Oslo': {'Berlin': 1, 'Moscow': 3, 'Reykjavik': 5},
#              'Moscow': {'Belgrade': 5, 'Athens': 4, 'Oslo': 3},
#              'London': {'Reykjavik': 4},
#              'Rome': {'Berlin': 2, 'Athens': 2},
#              'Berlin': {'Oslo': 1, 'Rome': 2},
#              'Belgrade': {'Moscow': 5, 'Athens': 1},
#              'Athens': {'Belgrade': 1, 'Moscow': 4, 'Rome': 2}}

class Graph(object):
    def __init__(self, nodes, init_graph): # __init__ выполняется всегда в начале присвоения класса nodes - список, init_graph - словарь
        self.nodes = nodes # присваиваем список с именами городов
        self.graph = init_graph # присваиваем сконструированный нами граф
# self используется для вызова переменных вне одной функции в классе, сейчас присваиваем, потом используем в других функциях(методах) у класса        
    def get_nodes(self):
        #Выводит вершины графа
        return self.nodes
    
    def get_neighbours(self, node):
        #Выводит соседей вершины
        connections = [] # Словарь с соседями
        for out_node in self.nodes: # Перебираем все узлы
            if self.graph[node].get(out_node, False) != False: # Если для выбранной вершины node есть out_node выводится числовое значение пути,если нет - False как указано в .get()
                connections.append(out_node) # И если путь есть записываем его в список соседей
        return connections
    
    def value(self, node1, node2):
        #Выводит значение ребра между двумя вершинами
        return self.graph[node1][node2]

# Создав класс обращения к графу(нашему списку) реализуем алгоритм 
def dijkstra_algorithm(graph, start_node): # Он принимает наш словарь под классом graph и начальный узел
    unvisited_nodes = list(graph.get_nodes()) # Записывает непосещённые узлы как все вершины графа для начала, чтобы далее убирать уже посещённые узлы
 
    # Словарь для обозначения минимального пути на каждый узел
    shortest_path = {}
 
    # Словарь для получения предыдущего узла из которого мы пришли(после мы выведем пути от нужного до начального и перевернём список - это и будет наш путь в названиях городов)
    previous_nodes = {}
 
    # Мы будем использовать max_value для инициализации значения "бесконечности" непосещенных узлов
    max_value = sys.maxsize
    for node in unvisited_nodes: # Для каждой вершины мы сохраняем значение нашей "бесконечности" чтобы потом заменить их на минимальные пути 
        shortest_path[node] = max_value
    # Значение начального узла - 0  
    shortest_path[start_node] = 0
    
    # Алгоритм выполняется до тех пор, пока мы не посетим все узлы
    while unvisited_nodes:
        # Приведенный ниже блок кода находит узел который уже посчитан и может суммироваться с остальными путями
#         print(unvisited_nodes)
        current_min_node = None
        for node in unvisited_nodes: # Пробегаемся по всем узлам
            if current_min_node == None: # Если минимального нету
                current_min_node = node # Присваиваем первый попавшийся
            elif shortest_path[node] < shortest_path[current_min_node]: # Если кратчайший записанный путь от нашего node меньше кратчайшего от минимального на данный момент
                current_min_node = node # То присваиваем минимуму на данный момент node, пробегаемый по всем не посещённым значениям
                
#         print(current_min_node)
        # Приведенный ниже блок кода извлекает соседей текущего узла и обновляет их расстояния
        neighbors = graph.get_neighbours(current_min_node) # Берём список соседей от нашего "минимального" узла
        for neighbor in neighbors: # Пробегаемся по всем этим значениям
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)#Считаем для каждого соседа сумму путей
            if tentative_value < shortest_path[neighbor]: # При этом если посчитанный путь меньше чем уже записанный в других итерациях то 
                shortest_path[neighbor] = tentative_value # Этот минимальный на данный момент записывается в кратчайшие пути под именем соседа к которому мы считали
                previous_nodes[neighbor] = current_min_node # И при этом заполняем словарь для будущего отображения пути
 
        # После посещения его соседей мы отмечаем узел как "посещенный", тоесть удаляем его
        unvisited_nodes.remove(current_min_node)
    
    return previous_nodes, shortest_path

def print_result(previous_nodes, shortest_path, start_node, target_node): # Создаём функцию для отображения нашего пути и суммы минимальных путей до цели
    path = [] # Сам путь
    node = target_node # Конечная цель
    
    while node != start_node: # И с конца по начало, пока node не стартовый узел
        path.append(node)
        node = previous_nodes[node]
 
   # Добавить начальный узел вручную
    path.append(start_node)
    
    print("Найден следующий лучший маршрут с ценностью {}.".format(shortest_path[target_node]))
    print(" -> ".join(reversed(path)))

nodes = ["Reykjavik", "Oslo", "Moscow", "London", "Rome", "Berlin", "Belgrade", "Athens"]
 
init_graph = {}
for node in nodes:
    init_graph[node] = {}
    
init_graph["Reykjavik"]["Oslo"] = 10
init_graph["Reykjavik"]["London"] = 25
init_graph["Oslo"]["Berlin"] = 13
init_graph["Oslo"]["Moscow"] = 4
init_graph["Moscow"]["Belgrade"] = 34
init_graph["Moscow"]["Athens"] = 26
init_graph["Athens"]["Belgrade"] = 10
init_graph["Rome"]["Berlin"] = 7
init_graph["Rome"]["Athens"] = 9

graph = Graph(nodes, init_graph)

previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node="Reykjavik")

print_result(previous_nodes, shortest_path, start_node="Reykjavik", target_node="Belgrade")

['Reykjavik', 'Oslo', 'Moscow', 'London', 'Rome', 'Berlin', 'Belgrade', 'Athens']
Reykjavik
['Oslo', 'Moscow', 'London', 'Rome', 'Berlin', 'Belgrade', 'Athens']
Oslo
['Moscow', 'London', 'Rome', 'Berlin', 'Belgrade', 'Athens']
Moscow
['London', 'Rome', 'Berlin', 'Belgrade', 'Athens']
Berlin
['London', 'Rome', 'Belgrade', 'Athens']
London
['Rome', 'Belgrade', 'Athens']
Athens
['Rome', 'Belgrade']
Belgrade
['Rome']
Rome
Найден следующий лучший маршрут с ценностью 48.
Reykjavik -> Oslo -> Moscow -> Belgrade


In [9]:
shortest_path

{'Reykjavik': 0,
 'Oslo': 10,
 'Moscow': 14,
 'London': 25,
 'Rome': 30,
 'Berlin': 23,
 'Belgrade': 48,
 'Athens': 39}

In [10]:
previous_nodes

{'Oslo': 'Reykjavik',
 'London': 'Reykjavik',
 'Moscow': 'Oslo',
 'Berlin': 'Oslo',
 'Belgrade': 'Moscow',
 'Athens': 'Rome',
 'Rome': 'Berlin'}

In [7]:
init_graph

{'Reykjavik': {'Oslo': 10, 'London': 25},
 'Oslo': {'Berlin': 13, 'Moscow': 4, 'Reykjavik': 10},
 'Moscow': {'Belgrade': 34, 'Athens': 26, 'Oslo': 4},
 'London': {'Reykjavik': 25},
 'Rome': {'Berlin': 7, 'Athens': 9},
 'Berlin': {'Oslo': 13, 'Rome': 7},
 'Belgrade': {'Moscow': 34, 'Athens': 10},
 'Athens': {'Belgrade': 10, 'Moscow': 26, 'Rome': 9}}

In [8]:
graph.graph

{'Reykjavik': {'Oslo': 10, 'London': 25},
 'Oslo': {'Berlin': 13, 'Moscow': 4, 'Reykjavik': 10},
 'Moscow': {'Belgrade': 34, 'Athens': 26, 'Oslo': 4},
 'London': {'Reykjavik': 25},
 'Rome': {'Berlin': 7, 'Athens': 9},
 'Berlin': {'Oslo': 13, 'Rome': 7},
 'Belgrade': {'Moscow': 34, 'Athens': 10},
 'Athens': {'Belgrade': 10, 'Moscow': 26, 'Rome': 9}}